## Functions

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

In [26]:
def get_ablation_data(problem_name, iterations = 1000):
    algs_list = ["ACO", "MMACO", "ACOSGD", "ADACO", "ACOPG", "ACOPPO"]
    
    file_template = ""
    files = [
             {"name": alg, "replay": size, "file": f"experiments/ablation/results/{problem_name}_{alg}_{size}_{iterations}.npy"}
                for alg in algs_list 
                for size in ["none", "small", "big",]
            ]
    
    for f in files:
        try:
            results = (np.load(f["file"]))
            
        except:
            print("Failed to find: " + f["file"])
            continue
            
        for i in range(results.shape[0]):
            f[f"results{i}"] = results[i]
            f[f"min_results{i}"] = float(results[i].min())
        f["min"] = results.min()
        f["min_avg"] = np.mean([results[i].min() for i in range(results.shape[0])])
        f["std"] = np.std([results[i].min() for i in range(results.shape[0])])
            
    dat = pd.DataFrame(files)

    dat = run_ablation_stats(dat)

    return dat


def get_comparative_data(problem_name, iterations = 1000):
    algs_list = ["ACO", "MMACO", "ANTQ", "ACOSGD", "ADACO", "ACOPG", "ACOPPO"]
    
    file_template = ""
    files = [
             {"name": alg, "file": f"experiments/comparative/results/{problem_name}_{alg}_{iterations}.npy"}
                for alg in algs_list 
            ]
    
    for f in files:
        try:
            results = (np.load(f["file"]))
            
        except:
            print("Failed to find: " + f["file"])
            continue
            
        for i in range(results.shape[0]):
            f[f"results{i}"] = results[i]
            f[f"min_results{i}"] = float(results[i].min())
        f["min"] = results.min()
        f["min_avg"] = np.mean([results[i].min() for i in range(results.shape[0])])
        f["std"] = np.std([results[i].min() for i in range(results.shape[0])])
            
    dat = pd.DataFrame(files)
    dat = run_comparative_stats(dat)

    return dat

In [3]:
def wilcox(dat):
    for alg in np.unique(dat["name"]):
        stat, p_value = wilcoxon(none, small)
        print(f"Statistic: {stat}, P-value: {p_value}")

def run_ablation_stats(dat):
    for alg in np.unique(dat["name"]):
        # Calculate the values for different replay types
        none_pt = dat.loc[(dat["name"] == alg) & (dat["replay"] == "none"), "min_avg"].values
        small_std = dat.loc[(dat["name"] == alg) & (dat["replay"] == "small"), "std"].values
        small_pt = dat.loc[(dat["name"] == alg) & (dat["replay"] == "small"), "min_avg"].values
        big_std = dat.loc[(dat["name"] == alg) & (dat["replay"] == "big"), "std"].values
        big_pt = dat.loc[(dat["name"] == alg) & (dat["replay"] == "big"), "min_avg"].values
    
        if none_pt.size > 0:  # Ensure none_pt is not empty
            none_pt = none_pt[0]
            
            # Assign percentages to the 'perc' column
            dat.loc[(dat["name"] == alg) & (dat["replay"] == "small"), "perc"] = 100 * (none_pt - small_pt) / none_pt
            dat.loc[(dat["name"] == alg) & (dat["replay"] == "big"), "perc"] = 100* (none_pt - big_pt) / none_pt
    return dat

def run_comparative_stats(dat):
    aco_pt = dat.loc[(dat["name"] == "ACO"), "min_avg"].values
    for alg in np.unique(dat["name"]):
        alg_pt = dat.loc[(dat["name"] == alg), "min_avg"].values
        dat.loc[(dat["name"] == alg), "perc"] = 100 * (aco_pt - alg_pt) / aco_pt
    return dat

## Ablation Data

In [17]:
pr76_dat = get_ablation_data("pr76")
pr76_dat[["name", "replay", "perc"]]


,name,replay,perc
0,ACO,none,NaN
1,ACO,small,0.171337
2,ACO,big,0.618383
3,MMACO,none,NaN
4,MMACO,small,1.795789
5,MMACO,big,1.454393
6,ACOSGD,none,NaN
7,ACOSGD,small,2.455283
8,ACOSGD,big,3.064940
9,ADACO,none,NaN


In [5]:
df = pr76_dat
algorithm = "ACO"

none = df[(df['name'] == algorithm) & (df['replay'] == 'none')].iloc[:, 2:7].values.flatten()
small = df[(df['name'] == algorithm) & (df['replay'] == 'small')].iloc[:, 2:7].values.flatten()
big = df[(df['name'] == algorithm) & (df['replay'] == 'big')].iloc[:, 2:7].values.flatten()

print(none)
print(small)
print(big)

['experiments/ablation/results/pr76_ACO_none_1000.npy'
 array([839294.24008464, 676481.0800872 , 660842.6925505 , ...,
        707085.10245373, 689733.15063456, 705011.99565453])
 651569.3759813269
 array([808575.58907403, 669417.15062725, 662361.43168357, ...,
        711094.93813973, 704302.32238123, 703546.43327102])
 651388.9085323925]
['experiments/ablation/results/pr76_ACO_small_1000.npy'
 array([823991.46330668, 661818.56725237, 661818.56725237, ...,
        653108.12225785, 653108.12225785, 653108.12225785])
 653108.1222578471
 array([808575.58907403, 643677.90189055, 643677.90189055, ...,
        643677.90189055, 643677.90189055, 643677.90189055])
 643677.901890554]
['experiments/ablation/results/pr76_ACO_big_1000.npy'
 array([818956.68824233, 656018.1246146 , 656018.1246146 , ...,
        647955.57110286, 647955.57110286, 647955.57110286])
 647955.5711028557
 array([805666.67400877, 663027.89264541, 663027.89264541, ...,
        643434.82219536, 643434.82219536, 643434.822195

In [6]:
att48_dat = get_ablation_data("att48")
att48_dat[["name", "replay", "min", "min_avg", "perc"]]

,name,replay,min,min_avg,perc
0,ACO,none,217505.925094,218714.959884,NaN
1,ACO,small,213420.924080,216072.903468,1.207991
2,ACO,big,215021.331846,217676.234410,0.474922
3,MMACO,none,215318.475026,215846.984923,NaN
4,MMACO,small,210831.323158,211501.067166,2.013425
5,MMACO,big,208881.781793,210831.243396,2.323749
6,ACOSGD,none,204913.870674,207503.619901,NaN
7,ACOSGD,small,201802.202123,203794.908358,1.787300
8,ACOSGD,big,203264.668155,203609.116993,1.876836
9,ADACO,none,204718.545268,205481.739772,NaN


In [7]:
rd100_dat = get_ablation_data("rd100")
rd100_dat[["name", "replay", "min", "min_avg", "perc"]]

Failed to find: experiments/ablation/results/rd100_MMACO_none_1000.npy
Failed to find: experiments/ablation/results/rd100_MMACO_small_1000.npy


,name,replay,min,min_avg,perc
0,ACO,none,56655.332803,57227.826971,NaN
1,ACO,small,57112.246116,57476.694470,-0.434871
2,ACO,big,56105.217439,56799.229319,0.748932
3,MMACO,none,NaN,NaN,NaN
4,MMACO,small,NaN,NaN,NaN
5,MMACO,big,55195.485086,55579.201518,NaN
6,ACOSGD,none,54152.864608,54358.600506,NaN
7,ACOSGD,small,52739.758844,53101.094156,2.313353
8,ACOSGD,big,51770.443273,52169.657145,4.026857
9,ADACO,none,53344.393422,53447.489185,NaN


In [8]:
tsp225_dat = get_ablation_data("tsp225", iterations = 2000)
tsp225_dat[["name", "replay", "min", "min_avg", "perc"]]

Failed to find: experiments/ablation/results/tsp225_MMACO_none_2000.npy
Failed to find: experiments/ablation/results/tsp225_MMACO_small_2000.npy
Failed to find: experiments/ablation/results/tsp225_MMACO_big_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOSGD_none_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOSGD_small_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOSGD_big_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOPG_none_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOPG_small_2000.npy
Failed to find: experiments/ablation/results/tsp225_ACOPG_big_2000.npy


,name,replay,min,min_avg,perc
0,ACO,none,65398.995251,65592.286050,NaN
1,ACO,small,64984.377662,65172.403974,0.640139
2,ACO,big,64870.551606,65174.980706,0.636211
3,MMACO,none,NaN,NaN,NaN
4,MMACO,small,NaN,NaN,NaN
5,MMACO,big,NaN,NaN,NaN
6,ACOSGD,none,NaN,NaN,NaN
7,ACOSGD,small,NaN,NaN,NaN
8,ACOSGD,big,NaN,NaN,NaN
9,ADACO,none,62217.011439,62533.518588,NaN


## Comparative Data

In [27]:
att48_comp_dat = get_comparative_data("att48", iterations = 1000)
att48_comp_dat[["name", "min", "min_avg", "perc"]]

,name,min,min_avg,perc
0,ACO,215068.717278,217307.854594,0.000000
1,MMACO,208327.581189,211999.684242,2.442696
2,ANTQ,215068.717278,217307.854594,0.000000
3,ACOSGD,202379.663814,203621.466111,6.298156
4,ADACO,204130.029117,205688.091617,5.347144
5,ACOPG,199643.209790,201289.690319,7.371185
6,ACOPPO,199183.571695,200534.504006,7.718704


In [28]:
try:
    pr76_comp_dat = get_comparative_data("pr76", iterations = 2000)
    display(pr76_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

,name,min,min_avg,perc
0,ACO,637338.550045,648598.292224,0.000000
1,MMACO,623290.545669,632246.143338,2.521152
2,ANTQ,637338.550045,648598.292224,0.000000
3,ACOSGD,594191.522751,601968.498908,7.189318
4,ADACO,604530.379377,609426.944975,6.039385
5,ACOPG,587883.813177,592670.373178,8.622890
6,ACOPPO,580065.695265,588047.763027,9.335598


In [36]:
try:
    rd100_comp_dat = get_comparative_data("rd100", iterations = 2000)
    display(rd100_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

,name,min,min_avg,perc
0,ACO,55674.034934,57067.838298,0.000000
1,MMACO,54448.135845,55271.438214,3.147833
2,ANTQ,55674.034934,57067.838298,0.000000
3,ACOSGD,52692.832145,52960.877546,7.196629
4,ADACO,53025.754533,53352.774864,6.509907
5,ACOPG,51268.913999,51751.789185,9.315315
6,ACOPPO,51200.114995,51501.581210,9.753755


In [30]:
try:
    tsp225_comp_dat = get_comparative_data("tsp225", iterations = 5000)
    display(tsp225_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

Failed to find: experiments/comparative/results/tsp225_ACO_5000.npy
Failed to find: experiments/comparative/results/tsp225_MMACO_5000.npy
Failed to find: experiments/comparative/results/tsp225_ANTQ_5000.npy
Failed to find: experiments/comparative/results/tsp225_ACOSGD_5000.npy
Failed to find: experiments/comparative/results/tsp225_ADACO_5000.npy
Failed to find: experiments/comparative/results/tsp225_ACOPG_5000.npy
Failed to find: experiments/comparative/results/tsp225_ACOPPO_5000.npy
no data yet


In [31]:
try:
    pcb442_comp_dat = get_comparative_data("pcb442", iterations = 5000)
    display(pcb442_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

Failed to find: experiments/comparative/results/pcb442_ACO_5000.npy
Failed to find: experiments/comparative/results/pcb442_MMACO_5000.npy
Failed to find: experiments/comparative/results/pcb442_ANTQ_5000.npy
Failed to find: experiments/comparative/results/pcb442_ACOSGD_5000.npy
Failed to find: experiments/comparative/results/pcb442_ADACO_5000.npy
Failed to find: experiments/comparative/results/pcb442_ACOPG_5000.npy
Failed to find: experiments/comparative/results/pcb442_ACOPPO_5000.npy
no data yet


In [32]:
try:
    att532_comp_dat = get_comparative_data("att532", iterations = 5000)
    display(att532_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

Failed to find: experiments/comparative/results/att532_ACO_5000.npy
Failed to find: experiments/comparative/results/att532_MMACO_5000.npy
Failed to find: experiments/comparative/results/att532_ANTQ_5000.npy
Failed to find: experiments/comparative/results/att532_ACOSGD_5000.npy
Failed to find: experiments/comparative/results/att532_ADACO_5000.npy
Failed to find: experiments/comparative/results/att532_ACOPG_5000.npy
Failed to find: experiments/comparative/results/att532_ACOPPO_5000.npy
no data yet


In [33]:
try:
    ali535_comp_dat = get_comparative_data("ali535", iterations = 5000)
    display(ali535_comp_dat[["name", "min", "min_avg", "perc"]])
except:
    print("no data yet")

Failed to find: experiments/comparative/results/ali535_ACO_5000.npy
Failed to find: experiments/comparative/results/ali535_MMACO_5000.npy
Failed to find: experiments/comparative/results/ali535_ANTQ_5000.npy
Failed to find: experiments/comparative/results/ali535_ACOSGD_5000.npy
Failed to find: experiments/comparative/results/ali535_ADACO_5000.npy
Failed to find: experiments/comparative/results/ali535_ACOPG_5000.npy
Failed to find: experiments/comparative/results/ali535_ACOPPO_5000.npy
no data yet
